In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv(r'E:\PythonCore\datafiles\SMSSpamCollection', 
                 sep="\t", header=None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
df.head()

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [54]:
from sklearn.feature_extraction.text import CountVectorizer

In [67]:
sw = set({"this", "is", "my"})

In [69]:
sw

{'is', 'my', 'this'}

In [70]:
cv = CountVectorizer(ngram_range=(1, 3), stop_words=sw)

In [71]:
sample_text = ['This is my first sentence', 
               'This is my second sentence', 
               'This is my third sentence']

In [72]:
count_matrix = cv.fit_transform(sample_text)

In [73]:
type(count_matrix)

scipy.sparse.csr.csr_matrix

In [74]:
count_matrix.toarray()

array([[1, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 1, 1]], dtype=int64)

In [75]:
cv.get_feature_names()

['first',
 'first sentence',
 'second',
 'second sentence',
 'sentence',
 'third',
 'third sentence']

In [76]:
df_test = pd.DataFrame(count_matrix.toarray(), 
                       columns=cv.get_feature_names())
df_test

,first,first sentence,second,second sentence,sentence,third,third sentence
0,1,1,0,0,1,0,0
1,0,0,1,1,1,0,0
2,0,0,0,0,1,1,1


In [88]:
X  = df.iloc[:,1].values
y = df.iloc[:,0].values
print(X.shape, y.shape)

(5572,) (5572,)


In [89]:
y

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

In [90]:
X

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [146]:
from sklearn.naive_bayes import MultinomialNB

In [147]:
cv1= CountVectorizer(stop_words='english')
X_terms = cv1.fit_transform(X)
X_terms

<5572x8444 sparse matrix of type '<class 'numpy.int64'>'
	with 43578 stored elements in Compressed Sparse Row format>

In [148]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X_terms,y,test_size=0.2, random_state=64)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4457, 8444) (1115, 8444) (4457,) (1115,)


In [150]:
model = MultinomialNB()
model.fit(X_train, y_train)
yhat = model.predict(X_test)
accuracy_score(y_test, yhat)

0.979372197309417

In [151]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

         ham       0.99      0.98      0.99       986
        spam       0.88      0.95      0.91       129

    accuracy                           0.98      1115
   macro avg       0.94      0.96      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [152]:
test_mail =[''' Dear Sir, you have won a lottery of USD 100000. Contact a,b@yahoo.com for
                    details''',
           '''Dear Sir, please be for the meeting on specified time''',
           '''Would you like a package holiday. You have won 1st prize in quiz competition''']

In [153]:
model.predict(cv1.transform(test_mail))

array(['spam', 'ham', 'spam'], dtype='<U4')